In [1]:
import dask.dataframe as ddf
import pandas as pd
import numpy as np
from infomap import Infomap
import networkx as nx

/home/ubuntu/.local/lib/python3.8/site-packages/dask/dataframe/utils.py:374: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/home/ubuntu/.local/lib/python3.8/site-packages/dask/dataframe/utils.py:374: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/home/ubuntu/.local/lib/python3.8/site-packages/dask/dataframe/utils.py:374: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)


In [2]:
sim_matrix = pd.read_parquet('result/user-similarity')

In [4]:
sim = sim_matrix.values

In [5]:
sim = sim - 0.01

In [6]:
editors = list(sim_matrix.index)

editor_info = pd.DataFrame(editors).reset_index().rename(columns={'index':'name',0:'contributor.username'})

res = pd.DataFrame()

In [9]:
def param_test(threshold, method):
    if method == 'sin':
        dis = np.arccos(sim)/pi
    else:
        dis = 1 - sim
    G = nx.from_numpy_matrix(dis) 
    edge_weights = nx.get_edge_attributes(G,'weight')
    G.remove_edges_from((e for e, w in edge_weights.items() if w > threshold))
    im = Infomap(silent=True, num_trials=10,include_self_links=False)   
    mapping = im.add_networkx_graph(G)
    im.run()
    df = im.get_dataframe(columns=["name","node_id", "module_id"])
    num_modules = len(df['module_id'].unique())
    a = df.groupby('module_id').count()
    num_singe_editor_groups = len(a[a['name'] == 1])
    max_ = a['name'].max()
    min_ = a['name'].min()
    greater_than_50 = len(a[a['name']>50])
    ans = 'threshold is {}, number of groups is {}, number of single editor groups is {}, '
    ans += 'max group contains {} editors, {} groups contains more than 50 editors'
    print(ans.format(threshold,num_modules,num_singe_editor_groups,max_,greater_than_50))
    
    fin = df.merge(editor_info,on='name')
    fin = fin.rename(columns={'module_id':'group'}).set_index('contributor.username')[['group']]
    res['threshold-' + str(threshold)] = fin['group']

In [10]:
test = np.arange(0,1.1,0.1).round(2)

In [11]:
for i in test:
    b = param_test(i,'original')

threshold is 0.0, number of groups is 4631, number of single editor groups is 4631, max group contains 1 editors, 0 groups contains more than 50 editors
threshold is 0.1, number of groups is 4517, number of single editor groups is 4438, max group contains 7 editors, 0 groups contains more than 50 editors
threshold is 0.2, number of groups is 4374, number of single editor groups is 4234, max group contains 11 editors, 0 groups contains more than 50 editors
threshold is 0.3, number of groups is 4181, number of single editor groups is 3979, max group contains 14 editors, 0 groups contains more than 50 editors
threshold is 0.4, number of groups is 3715, number of single editor groups is 3441, max group contains 72 editors, 1 groups contains more than 50 editors
threshold is 0.5, number of groups is 3027, number of single editor groups is 2745, max group contains 315 editors, 6 groups contains more than 50 editors
threshold is 0.6, number of groups is 2057, number of single editor groups is

In [13]:
res.to_csv('result/all-data-cluster-param-test-by-revert-graph.csv',encoding='utf-16', sep="\t")